In [5]:
import tdmclient.notebook
await tdmclient.notebook.start()


NodeLockError: Node lock error (current status: busy)

In [6]:
import time
import sys
import cv2
import pygame as pg
import numpy as np
from importlib import reload

import motion_control
import vision
import kalman
import AlgoGlobNav as gn

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
reload(motion_control)
reload(vision)
reload(kalman)
reload(gn)

In [9]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    """
    Sets the motor speeds of the Thymio
    param l_speed: left motor speed
    param r_speed: right motor speed
    """
    global motor_left_target, motor_right_target

    motor_left_target = l_speed
    motor_right_target = r_speed

In [10]:
@tdmclient.notebook.sync_var
def read_motors():
    """
    Read the motor speeds of the Thymio
    """
    global motor_left_speed, motor_right_speed

    return [motor_left_speed, motor_right_speed]


In [11]:
@tdmclient.notebook.sync_var
def read_proximity():
    """
    Read the proximity sensors of the Thymio
    """
    global prox_horizontal

    return prox_horizontal

## Vision

In [ ]:
vid = cv2.VideoCapture(1)

In [12]:
%%run_python
nf_leds_prox_h(0,0,0,0,0,0,0,0)
nf_leds_prox_v(0,0)
nf_leds_rc(0)
nf_leds_sound(0)
nf_leds_temperature(0,0)
nf_leds_top(0,0,0)
nf_leds_bottom_left(0,0,0)
nf_leds_bottom_right(0,0,0)
nf_leds_buttons(30,0,0,0)
nf_leds_top(0,255,0)


## Kalman Filter

The state of the Thymio is estimated using a Kalman filter. The choice for a kalman filter
was made because we decided for the global navigation to use nodes described in a coordinate system
with x and y coordinates. Thus, also the position of the Thymio should be estimated in that coordinate system.
Since we have multiple sensor readings: vision and wheel encoder, the Kalman filter was selected to fuse
these measurements. 

Since the position dynamics are independent of the orientation dynamics, two Kalman filters are used. Both are 
initialized using a vision measurement.

### Position Kalman Filter
The dynamic system used for the position estimation:

$$
\begin{bmatrix}
x_{k+1} \\ y_{k+1} \\ v_{x_{k+1}} \\ v_{y_{k+1}}
\end{bmatrix} =
\begin{bmatrix}
1 & 0 & T & 0 \\
0 & 1 & 0 & T \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
x_{k} \\ y_{k} \\ v_{x_{k}} \\ v_{y_{k}}
\end{bmatrix} + 
\begin{bmatrix}
\frac{T^2}{2} & 0 \\
0 & \frac{T^2}{2} \\
T & 0 \\
0 & T \\
\end{bmatrix}\boldsymbol{w}_k
$$

$$
y_{1_k} = \begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
x_{k} \\ y_{k} \\ v_{x_{k}} \\ v_{y_{k}}
\end{bmatrix} + \boldsymbol{v}_{k_1}
$$

$$
y_{2_k} = \begin{bmatrix}
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x_{k} \\ y_{k} \\ v_{x_{k}} \\ v_{y_{k}}
\end{bmatrix} + \boldsymbol{v}_{k_2}
$$

The $x$ and $y$ position coordinates are in meters and the $v_x$ and $v_y$ velocities are in m/s. The output measurement $y_{1_k}$ consists of the position and represents the vision measurements. Output measurement $y_{2_k}$ consists of the velocity and represents the wheel encoder measurements from which the Thymio's linear velocity is determined. Following covariance matrices have been defines:
$$
W = 
\begin{bmatrix}
\frac{T^2}{2} & 0 \\
0 & \frac{T^2}{2} \\
T & 0 \\
0 & T \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{acc}^2 & 0 \\ 
0 & \sigma_{acc}^2 \\
\end{bmatrix}
\begin{bmatrix}
\frac{T^2}{2} & 0 & T & 0 \\
0 & \frac{T^2}{2} & 0 & T
\end{bmatrix}
$$
$$
V_1 = 
\begin{bmatrix}
0.0001 & 0 \\
0 & 0.0001 \\
\end{bmatrix}
$$
$$
V_2 = 
\begin{bmatrix}
0.01 & 0 \\
0 & 0.01 \\
\end{bmatrix}
$$

The variance matrix $W$ corresponds to the process noise term $\boldsymbol{w}_k$ and the variance matrices $V_1$ and $V_2$ correspond to the output noise terms $\boldsymbol{v}_{k_1}$ and $\boldsymbol{v}_{k_2}$. The parameter $\sigma_{acc}$ represents the exptected standard deviation in linear acceleration. Since a constant velocity model is assumed, it accounts for model mismatch. $T$ is the time passed since the last estimate has been made.

The output covariance matrix $V_1$ was chosen to have 0.0001 on its diagonal since this corresponds to a standard deviation of 0.01m or an uncertainty of 1cm in the position measurement. The output covariance matrix $V_2$ was chose to have 0.01 on its diagonal since this corresponds to a standard deviation of 0.1m/s or an uncertainty of 10cm/s in the angular velocity measurement.


### Orientation Kalman Filter
The dynamic system used for the orientation estimation:

$$
\begin{bmatrix}
\theta_{k+1} \\ \omega_{k+1}
\end{bmatrix} =
\begin{bmatrix}
1 & T \\
0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
\theta_{k} \\ \omega_{k}
\end{bmatrix} + 
\begin{bmatrix}
\frac{T^2}{2} \\
T
\end{bmatrix}\boldsymbol{w}_k
$$

$$
y_{1_k} = \begin{bmatrix}
1 & 0 
\end{bmatrix}
\begin{bmatrix}
\theta_{k} \\ \omega_{k}
\end{bmatrix} + \boldsymbol{v}_{k_1}
$$

$$
y_{2_k} = \begin{bmatrix}
0 & 1
\end{bmatrix}
\begin{bmatrix}
\theta_{k} \\ \omega_{k}
\end{bmatrix} + \boldsymbol{v}_{k_2}
$$

The orientation angle $\theta$ is in radians and the angular velocity $\omega$ is in rad/s. The output measurement $y_{1_k}$ consists of the orientation and represents the vision measurements. Output measurement $y_{2_k}$ consists of the angular velocity and represents the wheel encoder measurements from which the Thymio's angular velocity is determined. Following covariance matrices have been defines:
$$
W = 
\begin{bmatrix}
\frac{T^2}{2} \\
 T \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{acc}^2
\end{bmatrix}
\begin{bmatrix}
\frac{T^2}{2} & T
\end{bmatrix}
$$
$$
V_1 = 
\begin{bmatrix}
0.0001
\end{bmatrix}
$$
$$
V_2 = 
\begin{bmatrix}
0.01
\end{bmatrix}
$$


The variance matrix $W$ corresponds to the process noise term $\boldsymbol{w}_k$ and the variance matrices $V_1$ and $V_2$ correspond to the output noise terms $\boldsymbol{v}_{k_1}$ and $\boldsymbol{v}_{k_2}$. The parameter $\sigma_{acc}$ represents the exptected standard deviation in angular acceleration. Since a constant angular velocity model is assumed, it accounts for model mismatch. $T$ is the time passed since the last estimate has been made.

The output covariance matrix $V_1$ was chosen to have 0.0001 since this corresponds to a standard deviation of 0.01rad or an uncertainty of 0.57deg in the orientation measurement. The output covariance matrix $V_2$ was chose to have 0.01 since this corresponds to a standard deviation of 0.1rad/s or an uncertainty of 5.7deg/s in the velocity measurement.

### Kalman filter update sequence
The estimate of Thymio's state is performed through the following sequence of calculations:
1. prediction step
- $\hat{x}_{k}^- = A\hat{x}_{k-1}^+$
- $P_k^- = AP_{k-1}^+A^T + W$

2. update step vision measurement (if available)
- $K_k = P_k^-C_{1}^T(C_1P_k^-C_1^T + V_1)^{-1}$
- $\hat{x}_k^+ = \hat{x}_k^- + K_k(y_{1_k}-C_{1}\hat{x}_k^-)$
- $P_k^+ = (I-K_kC_1)P_k^-$

3. update step wheel encoder (if available)
- $K_k = P_k^-C_{2}^T(C_1P_k^-C_2^T + V_2)^{-1}$
- $\hat{x}_k^+ = \hat{x}_k^- + K_k(y_{2_k}-C_{2}\hat{x}_k^-)$
- $P_k^+ = (I-K_kC_2)P_k^-$

There three step are executed at each timestep and are the same for the position as well as the orientation kalman filter

In [ ]:
# Kalman filter
t = time.time()
print("Start Kalman filter initialization")
mus_pos = []
mus_att = []

# Initialize state estimate using vision
try:
    pos, orientation = vision.fetchOdoMeters(vid, 1000)
except:
    print("Could not find initial state")

# Initialize kalman filters
try:
    kf_pos = kalman.Kalman(_acc_variance=0.5, _type="pose", init_x=np.hstack((pos,[0,0])))
    kf_att = kalman.Kalman(_acc_variance=0.5, _type="orientation", init_x=np.hstack((orientation,0)))
except:
    print("Could not initialize Kalman filters")

print("Initialization complete: %5.3f s" % (time.time()-t))
print("Initial pose:", kf_pos.mean)
print("Initial orientation", kf_att.mean*180/np.pi)
print("--------------------------------------")

## Global Navigation

In [1]:
# Global navigation
t = time.time()
print("Start global navigation initialization")

goal = vision.goalFetch(vid)
try:
    nodes, nodeCon, maskObsDilated, optimal_path = gn.opt_path(vid, goal)
except:
    print("Could not find optimal path")

print("Initialization complete: %5.3f s" % (time.time()-t))
print("--------------------------------------")

NameError: name 'time' is not defined

## Motion Control

In [ ]:
# Motion control
t = time.time()
print("Start motion control initialization")

mc = motion_control.MotionControl(optimal_path*100,
                                  kf_pos.mean[0]*100,
                                  kf_pos.mean[1]*100,
                                  kf_att.mean[0],
                                  nodes,
                                  nodeCon,
                                  maskObsDilated,
                                  goal)

print("Initialization complete: %5.3f s" % (time.time()-t))
print("--------------------------------------")

In [34]:
# Initialize pygame
pg.init()

h = int(841 / 2)
w = int(1189 / 2)
scr = pg.display.set_mode((w, h))

thymio = pg.image.load("thymio.png")
thymio = pg.transform.scale(thymio, (50, 50))

# main loop
running = True
simulate = False
local = False
while running:
    # Configure key presses for pygame and cv2 windows
    for event in pg.event.get():
        if event.type == pg.QUIT:
            cv2.destroyAllWindows()
            pg.quit()
            sys.exit()
        elif event.type == pg.KEYDOWN:
            if event.key == pg.K_RETURN :
                simulate = not simulate
    # Set up pygame window
    scr.fill((0, 0, 0))
    pg.draw.line(scr, (255, 255, 255), (0, int(h/2)), (w, int(h/2)))
    pg.draw.line(scr, (255, 255, 255), (int(w/2), 0), (int(w/2), h))

    # Start thymio movement
    if simulate:
        timer = time.time()
        # Save expected values and covariance matrices
        mus_pos.append(kf_pos.mean)
        mus_att.append(kf_att.mean)

        # Read sensor data
        try:
            pos, orientation = vision.fetchOdoMeters(vid)
            fetched = True
        except:
            fetched = False
            print("No vision measurements found")

        motor_speeds = read_motors()
        speed = (sum(motor_speeds)/2)*0.0004                                # magnitude of thymio's linear velocity [m/s]
        angular_speed = (motor_speeds[1]-motor_speeds[0])*0.0004/0.094      # magnitude of thymio's angular velocity [rad/s]
        vel = [speed*np.cos(mus_att[-1][0]), speed*np.sin(mus_att[-1][0])]

        # Estimate pose
        kalman.estimate_pose(time.time()-t, kf_pos, vel, pos, fetched)

        # Estimate orientation
        kalman.estimate_orientation(time.time()-t, kf_att, angular_speed, orientation, fetched)

        # Motion control
        prox_front = read_proximity()
        mc.update_motion(kf_pos.mean[0]*100, kf_pos.mean[1]*100, kf_att.mean[0], prox_front, vid)
        
        # Update motor speeds
        motors(l_speed=int(mc.l_speed), r_speed=int(mc.r_speed))

        # Draw Thymio and optimal path in window
        kalman.move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        kalman.drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)
        kalman.path(mc.optimal_path/100, w, h, scr)
    else:
        kalman.move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        kalman.drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)
        kalman.path(mc.optimal_path/100, w, h, scr)
        motors(0, 0)

    # Update pygame and cv2 windows
    optimal_path = mc.optimal_path/100
    try:
        output = vision.liveFeedback(vid, mc.nodes, mc.nodeCon, mc.maskObsDilated, optimal_path)
    except:
        pass
    else:
        cv2.imshow('Live Feedback', output)

    pg.display.flip()
    t = time.time()

print("===================================")
print("-------------Finished--------------")
print("===================================")

Start Kalman filter initialization
Initialization complete: 0.011 s
Initial pose: [0.11388542 0.67630417 0.         0.        ]
Initial orientation [14.44403572  0.        ]
--------------------------------------
Start global navigation initialization
0.2729017734527588
Initialization complete: 0.290 s
--------------------------------------
Start motion control initialization
Initialization complete: 0.001 s
--------------------------------------
0.1966688632965088
0.20326995849609375
0.15917611122131348
0.1736745834350586
0.2270505428314209
0.08504343032836914
0.23265504837036133
0.22202324867248535
0.27010202407836914
0.5249636173248291
0.6930866241455078
0.807349443435669
0.9972960948944092
1.2616875171661377
1.5304381847381592
1.7880191802978516
1.9897305965423584
2.1131319999694824
2.194668769836426
2.287302255630493
2.3904926776885986
2.5218474864959717
2.6128976345062256
2.704882860183716
3.011539936065674
3.549351930618286
No vision measurements found
No vision measurements fou

SystemExit: 

In [3]:
stop()